In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=ebc0718a52411c62045dc570ff0fd9d4044cda4ca6475943d1d46bc55832adf7
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [2]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import matplotlib.pyplot as plt
import time
from datetime import datetime

# CUDA kernel for dotplot calculation
cuda_kernel = """
__global__ void dotplot_kernel(unsigned char* seq1_chunk, unsigned char* seq2,
                                bool* result, int chunk_len, int seq1_len, int seq2_len,
                                int start_idx) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < chunk_len && col < seq2_len) {
        result[row * seq2_len + col] = (seq1_chunk[row] == seq2[col]);
    }
}
"""

def load_sequence(filename):
    """Load FASTA sequence and preprocess."""
    with open(filename, 'r') as file:
        seq = file.read()

    # Remove first line and line breaks
    seq = ''.join(seq.split('\n')[1:])

    # Create numpy array and map to integers
    mapping = {'A': 0, 'C': 1, 'G': 2, 'N': 3, 'T': 4}
    seq_array = np.array(list(seq))
    return np.vectorize(mapping.get)(seq_array).astype('uint8')

def cuda_dotplot_chunked(seq1, seq2, result_filename, num_chunks=10):
    """Perform CUDA-accelerated dotplot calculation with chunked processing."""
    seq1_len, seq2_len = len(seq1), len(seq2)

    # Create memory-mapped file for results
    result_map = np.memmap(result_filename, dtype='bool', mode='w+', shape=(seq1_len, seq2_len))

    # Calculate chunk size
    chunk_size = seq1_len // num_chunks

    # Allocate device memory for seq2 (constant across chunks)
    seq2_gpu = cuda.mem_alloc(seq2.nbytes)
    cuda.memcpy_htod(seq2_gpu, seq2)

    # Compile CUDA kernel
    mod = SourceModule(cuda_kernel)
    dotplot_kernel = mod.get_function("dotplot_kernel")

    # Setup grid and block dimensions
    block_size = 32
    grid_x = (seq2_len + block_size - 1) // block_size

    # Process seq1 in chunks
    for i in range(num_chunks):
        # Calculate chunk indices
        start_idx = i * chunk_size
        end_idx = start_idx + chunk_size if i != num_chunks - 1 else seq1_len

        # Current chunk
        seq1_chunk = seq1[start_idx:end_idx]
        chunk_len = len(seq1_chunk)

        # Allocate device memory for current chunk
        seq1_chunk_gpu = cuda.mem_alloc(seq1_chunk.nbytes)
        cuda.memcpy_htod(seq1_chunk_gpu, seq1_chunk)

        # Allocate device memory for chunk result
        result_chunk_gpu = cuda.mem_alloc(chunk_len * seq2_len * np.dtype('bool').itemsize)

        # Calculate grid dimensions for current chunk
        grid_y = (chunk_len + block_size - 1) // block_size

        # Launch kernel
        dotplot_kernel(
            seq1_chunk_gpu, seq2_gpu, result_chunk_gpu,
            np.int32(chunk_len), np.int32(seq1_len), np.int32(seq2_len),
            np.int32(start_idx),
            block=(block_size, block_size, 1),
            grid=(grid_x, grid_y)
        )

        # Allocate host buffer for this chunk
        result_chunk = np.zeros((chunk_len, seq2_len), dtype=bool)

        # Copy result back to host
        cuda.memcpy_dtoh(result_chunk, result_chunk_gpu)

        # Write chunk to memory-mapped file
        result_map[start_idx:end_idx, :] = result_chunk

    return result_map

def plot_dotplot(result_filename, seq1_len, seq2_len):
    """Plot the dot plot from memory-mapped file."""
    result_map = np.memmap(result_filename, dtype='bool', mode='r', shape=(seq1_len, seq2_len))
    plt.figure(figsize=(10, 10))
    plt.title("Dotplot CUDA")
    plt.xlabel("Seq2")
    plt.ylabel("Seq1")
    plt.imshow(result_map[:500, :500], cmap='binary', aspect='auto')
    plt.savefig("ResultadoCUDA.png")
    plt.close()

def main():
    # Start timing
    begin = time.time()
    print(datetime.today())

    # Load sequences
    seq1 = load_sequence('sample_data/archivos_dotplot/elemento1.fasta')
    seq2 = load_sequence('sample_data/archivos_dotplot/elemento2.fasta')

    # Result filename for memory mapping
    result_filename = './dotplot_result_cuda.dat'

    # Perform CUDA dotplot calculation with chunked memory processing
    result_map = cuda_dotplot_chunked(seq1, seq2, result_filename)

    # Plot result
    plot_dotplot(result_filename, len(seq1), len(seq2))

    # Cleanup (close the memory-mapped file)
    del result_map

    # End timing
    end = time.time()
    print(datetime.today())
    print(f"Tiempo de ejecución CUDA: {end-begin} segundos")

if __name__ == '__main__':
    main()

2024-11-29 23:40:01.130964
2024-11-29 23:42:42.544526
Tiempo de ejecución CUDA: 161.41354513168335 segundos
